# 🔍 RAG Pipeline For LLMs 🚀


## 📖 Project Overview

This Notebook implements a **Retrieval-Augmented Generation (RAG)** pipeline using state-of-the-art language models and vector databases. The system intelligently retrieves relevant information from Wikipedia articles and generates accurate, context-aware answers to user questions.



### ✨ Key Features

- 📚 **Dynamic Knowledge Retrieval** from Wikipedia
- 🧮 **Semantic Search** using sentence transformers
- ⚡ **Fast Vector Similarity** with FAISS indexing
- 🤖 **Intelligent Answer Generation** using pre-trained QA models
- 📊 **Confidence Scoring** for answer quality assessment
- 🎛️ **Customizable Parameters** (chunk size, retrieval count, etc.)

### 🏗️ Architecture

```
User Query → Embedding → FAISS Search → Retrieve Chunks → QA Model → Answer + Confidence
```

### 🛠️ Tech Stack & Models

**🔧 Core Technologies:**
- **🤗 Transformers**: For tokenization and question-answering models
- **📝 Sentence Transformers**: For semantic embeddings
- **⚡ FAISS**: For efficient vector similarity search
- **📖 Wikipedia API**: For knowledge base content
- **🐍 Python**: Core implementation language

**🤖 AI Models Used:**
- **📏 Text Chunking**: `sentence-transformers/all-mpnet-base-v2` tokenizer
- **🧮 Vector Embeddings**: `sentence-transformers/all-mpnet-base-v2` (768-dimensional)
- **❓ Question Answering**: `deepset/roberta-base-squad2` (RoBERTa fine-tuned on SQuAD 2.0)
- **🔍 Vector Search**: FAISS IndexFlatL2 for L2 distance similarity

## 📦 Installation & Setup

### 🔧 Installing Required Dependencies

First, let's install all the necessary Python packages for our RAG pipeline:

In [6]:
!pip install wikipedia
!pip install transformers
!pip install sentence-transformers
!pip install faiss-cpu
!pip install tf_keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 3.8 MB/s eta 0:00:00


### 📚 Importing Essential Libraries

Now let's import all the required libraries for our RAG implementation:

- **🔢 NumPy**: For numerical operations and array handling
- **📖 Wikipedia**: For fetching knowledge base content
- **🤗 Transformers**: For tokenization and QA models  
- **📝 Sentence Transformers**: For creating semantic embeddings
- **⚡ FAISS**: For efficient vector similarity search


In [2]:
import numpy as np
import wikipedia
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from sentence_transformers import SentenceTransformer
import faiss

## 📖 Retrieving Knowledge:

Fetch relevant Wikipedia articles based on a given topic:

In [12]:
def get_wikipedia_content(topic):
    try: 
        page = wikipedia.page(topic)
        return page.content
    except wikipedia.exceptions.PageError:
        return None
    except wikipedia.exceptions.DisambiguationError as e:
        # Handling cases where the topic is ambiguous:
        print(f"Ambiguous topic, Get more specific. Possible options: {e.options}")
        return None

# User Input:
topic = input("Enter a topic to learn about: ")
document = get_wikipedia_content(topic)

if not document:
    print("Could not find information on that topic.")
    exit()

## ✂️ Text Chunking:

Since Wikipedia articles can be long, we will split the text into smaller overlapping chunks for better retrieval:

- 📏 **Smart Segmentation**: Breaking text into manageable pieces (256 tokens each)
- 🔗 **Overlapping Chunks**: Maintaining context between segments (20 token overlap)
- ⚡ **Optimized Processing**: Avoiding token limit warnings and errors
- 🎯 **Better Retrieval**: Smaller chunks = more precise similarity matching

**Process:** Each chunk represents a focused piece of information that can be independently searched and retrieved for answering questions.

In [13]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")

def split_text(text, chunk_size = 256, chunk_overlap = 20):
    tokens = tokenizer.tokenize(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + chunk_size, len(tokens))
        chunks.append(tokenizer.convert_tokens_to_string(tokens[start:end]))
        if end == len(tokens):
            break
        start = end - chunk_overlap
    return chunks

chunks = split_text(document)
print(f"Total chunks: {len(chunks)}")

Token indices sequence length is longer than the specified maximum sequence length for this model (6573 > 512). Running this sequence through the model will result in indexing errors


Total chunks: 28


## 🧮 Vector Embeddings & Search Index:

### 🔄 Converting Text to Mathematical Vectors

Transforms human-readable text into numerical representations that machines can compare:

**🎯 The Process:**
- 📝 **Sentence Transformers**: Convert each text chunk into 768-dimensional vectors
- 🧠 **Semantic Encoding**: Capture meaning, not just keywords
- ⚡ **FAISS Indexing**: Store vectors for lightning-fast similarity search
- 📊 **L2 Distance**: Measure semantic similarity mathematically

To efficiently search for relevant chunks, we will use Sentence Transformers to convert text into embeddings and store them in a FAISS index:

In [14]:
embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embeddings = embedding_model.encode(chunks)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

## 🔍 Smart Query Processing & Retrieval

### ❓ From Question to Context

In the retrieval phase When a user asks a question, it will:


**The Retrieval Pipeline:**

1️⃣ **🎯 Query Embedding**: Convert user question to same vector space as knowledge chunks

2️⃣ **⚡ Similarity Search**: FAISS finds the k most semantically similar chunks  

3️⃣ **📋 Context Assembly**: Gather the most relevant information pieces

4️⃣ **🎛️ Customizable Retrieval**: Adjust k (number of chunks) based on needs


In [24]:
query = input("Ask a question about the topic: ")
query_embedding = embedding_model.encode([query])
query_embedding_array = np.array(query_embedding)

k = 3   # Number of chunks to retrieve
distances, indices = index.search(query_embedding_array, k)
retrieved_chunks = [chunks[i] for i in indices[0]]
print("Retrieved chunks: ")
for chunk in retrieved_chunks:
    print("- " + chunk)

Retrieved chunks: 
- ( portugal ), reims ( france ) and voros lobogo ( hungary ). the first european cup match took place on 4 september 1955, and ended in a 3 – 3 draw between sporting cp and partizan. the first goal in european cup history was scored by joao baptista martins of sporting cp. the inaugural final took place at the parc des princes between stade de reims and real madrid on 13 june 1956. the spanish squad came back from behind to win 4 – 3 thanks to goals from alfredo di stefano and marquitos, as well as two goals from hector rial. real madrid successfully defended the trophy next season in their home stadium, the santiago bernabeu, against fiorentina. after a scoreless first half, real madrid scored twice in six minutes to defeat the italians. in 1958, milan failed to capitalise after going ahead on the scoreline twice, only for real madrid to equalise. the final, held in heysel stadium, went to extra time where francisco gento scored the game - winning goal to allow rea

## 🤖 Intelligent Answer Generation

### 🧠 From Context to Precise Answers

The final step where retrieved knowledge becomes intelligent responses:

**🎯 Generation Process:**
- 🤖 **RoBERTa QA Model**: Pre-trained on SQuAD 2.0 dataset for optimal performance
- 📝 **Extractive Answering**: Finds exact answer spans within retrieved context
- 🎛️ **Context Fusion**: Combines multiple relevant chunks into unified context
- 📊 **Confidence Scoring**: Provides reliability metrics for each answer


**🔗 Complete RAG Flow**: `Wikipedia → Chunks → Embeddings → Retrieval → Generation → Answer + Confidence`

In [25]:
qa_model_name = "deepset/roberta-base-squad2"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer)

context = " ".join(retrieved_chunks)
answer = qa_pipeline(question = query, context = context)
print(f"Answer: {answer['answer']}")

Device set to use cpu


Answer: 15


### 📊 Answer Quality Assessment

Monitor the reliability and confidence of generated answers:


In [26]:
# Display confidence score
print(f"Confidence Score: {answer['score']:.4f} ({answer['score']*100:.1f}%)")


Confidence Score: 0.3357 (33.6%)
